In [1]:
import pandas as pd

In [2]:
apple_news_df = pd.read_pickle("./data/apple_news.pkl")
apple_news_df = apple_news_df.rename(columns={"date": "full_long_date_format"})
apple_news_df.head()

,full_long_date_format,headline,text,url
38,"Fri Apr 5, 2013 3:52pm EDT",Hedge fund manager Ackman says mistakes made i...,BOSTON (Reuters) - Hedge fund manager Willia...,http://www.reuters.com/article/2013/04/05/us-h...
155,"Tue Nov 13, 2012 4:41am EST",Microsoft and Google financials could surface ...,(Reuters) - Microsoft and Google's Motorola M...,http://www.reuters.com/article/2012/11/13/us-m...
189,"Tue Nov 13, 2012 8:32am EST","Microsoft's Windows head, once a possible CEO,...",SEATTLE (Reuters) - The executive most widel...,http://www.reuters.com/article/2012/11/13/us-m...
283,"Tue Feb 7, 2012 5:33pm EST",Wall St edges up in quiet day; Disney down late,NEW YORK (Reuters) - Stock index futures poi...,http://www.reuters.com/article/2012/02/07/us-m...
432,"Mon Jun 9, 2008 9:28pm EDT",iPhone to cut into AT&T earnings until 2010,NEW YORK (Reuters) - AT&T Inc will suffer lo...,http://www.reuters.com/article/2008/06/10/us-a...


In [3]:
from time import strptime
from datetime import datetime

s = "Fri Apr 5, 2013 3:52pm EDT"
s = s.replace(',', '')

date = s.split()[1:5]
month = strptime(date[0],'%b').tm_mon

date

['Apr', '5', '2013', '3:52pm']

In [4]:
x = datetime(int(date[2]), int(month), int(date[1]))
date_format = x.strftime("%Y-%m-%d")
date_format

'2013-04-05'

In [5]:
def cleanup_date_format(x):
    x = x.replace(',', '')
    date = x.split()[1:5]
    month = strptime(date[0],'%b').tm_mon
    x = datetime(int(date[2]), int(month), int(date[1]))
    date_format = x.strftime("%Y-%m-%d")
    return date_format

In [6]:
apple_news_df['Date'] = apple_news_df['full_long_date_format'].apply(lambda x: cleanup_date_format(x))
apple_news_df.head()

,full_long_date_format,headline,text,url,Date
38,"Fri Apr 5, 2013 3:52pm EDT",Hedge fund manager Ackman says mistakes made i...,BOSTON (Reuters) - Hedge fund manager Willia...,http://www.reuters.com/article/2013/04/05/us-h...,2013-04-05
155,"Tue Nov 13, 2012 4:41am EST",Microsoft and Google financials could surface ...,(Reuters) - Microsoft and Google's Motorola M...,http://www.reuters.com/article/2012/11/13/us-m...,2012-11-13
189,"Tue Nov 13, 2012 8:32am EST","Microsoft's Windows head, once a possible CEO,...",SEATTLE (Reuters) - The executive most widel...,http://www.reuters.com/article/2012/11/13/us-m...,2012-11-13
283,"Tue Feb 7, 2012 5:33pm EST",Wall St edges up in quiet day; Disney down late,NEW YORK (Reuters) - Stock index futures poi...,http://www.reuters.com/article/2012/02/07/us-m...,2012-02-07
432,"Mon Jun 9, 2008 9:28pm EDT",iPhone to cut into AT&T earnings until 2010,NEW YORK (Reuters) - AT&T Inc will suffer lo...,http://www.reuters.com/article/2008/06/10/us-a...,2008-06-09


In [7]:
apple_OHLCV = pd.read_csv('data/AAPL.csv') 
apple_OHLCV.head()

,Date,Open,High,Low,Close,Volume,Adj Close
0,2012-09-12,666.85,669.90,656.00,669.79,25410600,669.79
1,2012-09-11,665.11,670.10,656.50,660.59,17987400,660.59
2,2012-09-10,680.45,683.29,662.10,662.74,17428500,662.74
3,2012-09-07,678.05,682.48,675.77,680.44,11773800,680.44
4,2012-09-06,673.17,678.29,670.80,676.27,13971300,676.27


In [21]:
data = apple_OHLCV.merge(apple_news_df, on='Date')
data = data.drop_duplicates(subset=['Date'])
data = data.set_index('Date')
data.head(20)

,Open,High,Low,Close,Volume,Adj Close,full_long_date_format,headline,text,url
Date,,,,,,,,,,
2012-09-12,666.85,669.90,656.00,669.79,25410600,669.79,"Wed Sep 12, 2012 2:07pm EDT","Apple's iPhone 5 goes on sale Friday, starts a...",SAN FRANCISCO (Reuters) - Apple Inc will beg...,http://www.reuters.com/article/2012/09/12/us-a...
2012-09-11,665.11,670.10,656.50,660.59,17987400,660.59,"Tue Sep 11, 2012 7:57pm EDT",Dow climbs to highest since 2007,NEW YORK (Reuters) - The Dow industrials clo...,http://www.reuters.com/article/2012/09/11/us-m...
2012-09-10,680.45,683.29,662.10,662.74,17428500,662.74,"Mon Sep 10, 2012 7:47pm EDT",New iPhone could boost U.S. GDP by up to 0.5 p...,"(Reuters) - The next generation iPhone 5, whi...",http://www.reuters.com/article/2012/09/10/us-u...
2012-09-07,678.05,682.48,675.77,680.44,11773800,680.44,"Fri Sep 7, 2012 7:41pm EDT",Pandora shares plunge on fears of Apple service,(Reuters) - Apple Inc's potential launch of a...,http://www.reuters.com/article/2012/09/07/us-p...
2012-09-06,673.17,678.29,670.80,676.27,13971300,676.27,"Thu Sep 6, 2012 2:35am EDT","Sharp mortgages iPhone display plant, other Ja...",TOKYO (Reuters) - Japan's embattled Sharp Co...,http://www.reuters.com/article/2012/09/06/us-s...
2012-09-05,675.57,676.35,669.60,670.23,12013400,670.23,"Wed Sep 5, 2012 4:23pm EDT","Wall Street treads water, stays on ECB watch",NEW YORK (Reuters) - Stocks closed out a sec...,http://www.reuters.com/article/2012/09/05/us-m...
2012-09-04,665.76,675.14,664.50,674.97,13139000,674.97,"Tue Sep 4, 2012 6:22pm EDT",Wall Street cuts losses on Apple strength; Fed...,NEW YORK (Reuters) - The S&P 500 closed slig...,http://www.reuters.com/article/2012/09/04/us-m...
2012-08-30,670.64,671.55,662.85,663.87,10810700,663.87,"Thu Aug 30, 2012 12:07pm EDT",Amazon says Kindle Fire has sold out,(Reuters) - Amazon.com Inc said on Thursday i...,http://www.reuters.com/article/2012/08/30/net-...
2012-08-29,675.25,677.67,672.60,673.47,7243100,673.47,"Wed Aug 29, 2012 8:38am EDT",Apple/Samsung dispute not yet made it to China...,BEIJING (Reuters) - A patent dispute between...,http://www.reuters.com/article/2012/08/29/us-a...
